In [1]:
print('hello')

hello


In [2]:
! pip install llama_index==0.10.19 llama_index_core==0.10.19 torch llama-index-embeddings-huggingface peft optimum bitsandbytes

  Using cached llama_index-0.10.19-py3-none-any.whl.metadata (8.8 kB)
  Using cached llama_index_core-0.10.19-py3-none-any.whl.metadata (3.6 kB)
  Using cached torch-2.5.1-cp310-cp310-win_amd64.whl.metadata (28 kB)
  Using cached llama_index_embeddings_huggingface-0.4.0-py3-none-any.whl.metadata (767 bytes)
  Using cached peft-0.14.0-py3-none-any.whl.metadata (13 kB)
  Using cached optimum-1.23.3-py3-none-any.whl.metadata (20 kB)
  Using cached bitsandbytes-0.45.0-py3-none-win_amd64.whl.metadata (2.9 kB)
  Using cached llama_index_agent_openai-0.1.7-py3-none-any.whl.metadata (644 bytes)
  Using cached llama_index_cli-0.1.13-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_embeddings_openai-0.1.11-py3-none-any.whl.metadata (655 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.1.6-py3-none-any.whl.metadata (3.8 kB)
  Using cached llama_index_legacy-0.9.48.post4-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_llms_openai-0.1.31-py3-none-any.whl.me

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings,SimpleDirectoryReader,VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from transformers import AutoModelForCausalLM,AutoTokenizer

c:\Users\ajay2\OneDrive\Desktop\guvi\PROJECT_CAP\LLAMA RAG\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# To globally set the resources we are goin to use



In [5]:
Settings.embed_model=HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')
Settings.llm = None
Settings.chunk_size = 256 
Settings.chunk_overlap = 15 


LLM is explicitly disabled. Using MockLLM.


In [6]:
documents=SimpleDirectoryReader('content').load_data()

print(len(documents))


for doc in documents:
    if len(doc.text)== 0:
        documents.remove(doc)
        continue

print(len(documents))

48
48


# Creating a Vector Store

In [7]:
index= VectorStoreIndex.from_documents(documents)

# Set no of docs to retrive
top_k=2

# Configure retriver
retriever=VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)



# Assembling the Query engine


In [8]:
query_engine=RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)]
)

In [9]:
query="What's all  this text ablout?"

response=query_engine.query(query)

print(response)

context='Context: \n'

for i in range(top_k):
    context=context + response.source_nodes[i].text + '\n\n'

print(context)

Context information is below.
---------------------
page_label: 33
file_path: c:\Users\ajay2\OneDrive\Desktop\guvi\PROJECT_CAP\LLAMA RAG\content\2308.12950v3.pdf

0 0.0 0.0
freq. scaling 1/4 100.0 100.0 100.0 100.0 99.2 99.2 2.34 99.2 100.0 0.0 0.0 0.0
Ours (θ= 106) 95.3 95.3 100.0 100.0 95.3 100.0 54.7 100.0 98.4 3.1 85.9 85.9
Table 18: Function Key Retrieval Accuracy (%) Ablations . Ablation experiments are performed with
an earlier version of the 7B model; the last row refers to Code Llama 7B. All long context fine-tuning runs
employ a sequence length of 16,384 tokens.

page_label: 48
file_path: c:\Users\ajay2\OneDrive\Desktop\guvi\PROJECT_CAP\LLAMA RAG\content\2308.12950v3.pdf

The mathematical terms “quasi-prefunctoid”, “precategoroid” and “etalisation” do not exist and were invented
to make sure the model did not memorize. The model uses a plausible parametrization of “etalisations” of P
via morphisms fof the underlying “precategoroids” and invents a notation for the induced “eta

In [2]:
model_name='Qwen/Qwen2.5-0.5B-Instruct'

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=False,
    revision='main',
    #device_map='cuda:0' # we try to load on GPU
    )

# load tokenizer

tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

c:\Users\ajay2\OneDrive\Desktop\guvi\PROJECT_CAP\LLAMA RAG\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ajay2\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
prompt_templete_w_context= lambda context, query:  f"""you are an AI assistant tasked with answering question based on the provided PDF content. 
                              Please analyze the foolowing excerpt from the PDF and answer the question.  
                              PDf content:
                              {context}
                            Question:{query}
                            Instructions:
                            -Answer only based on the information provided in the PDF content above.
                            -if the answer cannot be found in the provided content.say "I cannot find the answer to the question and provide a pdf documents".
                            -Be concise and specifice.
                            -Include relevant quote or references from the PDF when applicable

                            Answer:"""

In [12]:
query="What is the long  context-finetuning ?"

prompt=prompt_templete_w_context(context,query)

inputs=tokenizer(prompt,return_tensors='pt')

outputs = model.generate(input_ids=inputs['input_ids'],max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

you are an AI assistant tasked with answering question based on the provided PDF content. 
                              Please analyze the foolowing excerpt from the PDF and answer the question.  
                              PDf content:
                              Context: 
0 0.0 0.0
freq. scaling 1/4 100.0 100.0 100.0 100.0 99.2 99.2 2.34 99.2 100.0 0.0 0.0 0.0
Ours (θ= 106) 95.3 95.3 100.0 100.0 95.3 100.0 54.7 100.0 98.4 3.1 85.9 85.9
Table 18: Function Key Retrieval Accuracy (%) Ablations . Ablation experiments are performed with
an earlier version of the 7B model; the last row refers to Code Llama 7B. All long context fine-tuning runs
employ a sequence length of 16,384 tokens.

The mathematical terms “quasi-prefunctoid”, “precategoroid” and “etalisation” do not exist and were invented
to make sure the model did not memorize. The model uses a plausible parametrization of “etalisations” of P
via morphisms fof the underlying “precategoroids” and invents a notation for the induc

In [14]:
pip install streamlit

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/9.1 MB ? e